In [1]:
import os

In [2]:
%pwd

'/Users/zeglazi/Documents/ETE_Chest_Cancer_ML_Project_MLFLOW_DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/zeglazi/Documents/ETE_Chest_Cancer_ML_Project_MLFLOW_DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentration: bool
    params_image_size: list 

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2025-06-06 00:52:07.677068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        create_directories([self.config.artifacts_root])
        
    
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest_data")
        create_directories([Path(training.root_dir)])
        
        training_config = TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentration= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )
        
        return training_config

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [22]:
class Training:
    def __init__(self, config):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def setup_data_generators(self):
        datagenerator_kwargs = dict(rescale=1./255)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        if self.config.params_is_augmentration:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        eval_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=os.path.join(self.config.training_data, "train"),
            shuffle=True,
            **dataflow_kwargs
        )

        self.test_generator = eval_datagenerator.flow_from_directory(
            directory=os.path.join(self.config.training_data, "test"),
            shuffle=False,
            **dataflow_kwargs
        )

        self.valid_generator = eval_datagenerator.flow_from_directory(
            directory=os.path.join(self.config.training_data, "valid"),
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.test_generator.samples // self.test_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.test_generator,
            validation_steps=self.validation_steps
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [23]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    
    training = Training(config=training_config)
    training.get_base_model()
    training.setup_data_generators()
    training.train()

except Exception as e:
    raise e

[2025-06-06 01:07:05,891: INFO: common : yaml file : config/config.yaml loaded successfully]
[2025-06-06 01:07:05,894: INFO: common : yaml file : params.yaml loaded successfully]
[2025-06-06 01:07:05,895: INFO: common : create directory at : artifacts]
[2025-06-06 01:07:05,896: INFO: common : create directory at : artifacts/training]
Found 343 images belonging to 2 classes.
Found 174 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Epoch 1/10


2025-06-06 01:07:06.613572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


21/21 [==============================] - ETA: 0s - loss: 17.1583 - accuracy: 0.5138

2025-06-06 01:07:37.384161: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


21/21 [==============================] - 46s 2s/step - loss: 17.1583 - accuracy: 0.5138 - val_loss: 10.8663 - val_accuracy: 0.2500
Epoch 2/10
21/21 [==============================] - 87s 4s/step - loss: 10.6344 - accuracy: 0.5719 - val_loss: 10.5935 - val_accuracy: 0.7500
Epoch 3/10
21/21 [==============================] - 68s 3s/step - loss: 5.4422 - accuracy: 0.6942 - val_loss: 22.0080 - val_accuracy: 0.2500
Epoch 4/10
21/21 [==============================] - 70s 3s/step - loss: 5.9076 - accuracy: 0.7003 - val_loss: 0.3545 - val_accuracy: 0.9312
Epoch 5/10
21/21 [==============================] - 71s 3s/step - loss: 3.6772 - accuracy: 0.7706 - val_loss: 2.6123 - val_accuracy: 0.8562
Epoch 6/10
21/21 [==============================] - 69s 3s/step - loss: 1.6253 - accuracy: 0.8593 - val_loss: 0.0304 - val_accuracy: 0.9937
Epoch 7/10
21/21 [==============================] - 73s 3s/step - loss: 1.9588 - accuracy: 0.8471 - val_loss: 0.3173 - val_accuracy: 0.9375
Epoch 8/10
21/21 [========